In [31]:
import psi4
import numpy as np
from psi4.driver.procrouting.response.scf_response import tdscf_excitations

In [32]:
mol = psi4.geometry("""
0 1
O
H 1 1.0
H 1 1.0 2 104.5
symmetry c1
""")

psi4.set_options({'basis':        'sto-3g',
                  'scf_type':     'pk',
                  'reference':    'rhf',
                  'mp2_type':     'conv',
                  'save_jk': True,
                  'e_convergence': 1e-8,
                  'd_convergence': 1e-8})

In [40]:
# compute the Hartree-Fock energy and wavefunction
scf_e, wfn = psi4.energy('SCF', return_wfn=True)
rpa = tdscf_excitations(wfn, states=4, tda=True)
print(rpa)

right vecs
[[ 1.11496908e-16 -5.29257371e-20]
 [-5.20723397e-17 -1.62756122e-17]
 [-8.97175766e-16  1.09154215e-16]
 [-1.54415002e-15  2.95588487e-17]
 [ 1.00000000e+00  4.76559551e-16]]
left vecs
[[ 1.11496908e-16 -5.29257371e-20]
 [-5.20723397e-17 -1.62756122e-17]
 [-8.97175766e-16  1.09154215e-16]
 [-1.54415002e-15  2.95588487e-17]
 [ 1.00000000e+00  4.76559551e-16]]
dipole
[[ 0.00000000e+00  0.00000000e+00  0.00000000e+00 -5.07919296e-02
   0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 -6.41172844e-01
   0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  9.08620836e-18
   0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [-5.07919296e-02 -6.41172844e-01  9.08620836e-18  0.00000000e+00
   0.00000000e+00 -2.87550614e-01 -2.87550614e-01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  

In [41]:
# Grab data from wavfunction

# number of doubly occupied orbitals
ndocc   = wfn.nalpha()

# total number of orbitals
nmo     = wfn.nmo()

# number of virtual orbitals
nvirt   = nmo - ndocc

# orbital energies
eps     = np.asarray(wfn.epsilon_a())

# occupied orbitals:
Co = wfn.Ca_subset("AO", "OCC")

# virtual orbitals:
Cv = wfn.Ca_subset("AO", "VIR")

In [42]:
# use Psi4's MintsHelper to generate ERIs
mints = psi4.core.MintsHelper(wfn.basisset())

# build the (ov|ov) integrals:
ovov = np.asarray(mints.mo_eri(Co, Cv, Co, Cv))

# build the (oo|vv) integrals:
oovv = np.asarray(mints.mo_eri(Co, Co, Cv, Cv))

In [43]:
# strip out occupied orbital energies, eps_o spans 0..ndocc-1
eps_o = eps[:ndocc]

# strip out virtual orbital energies, eps_v spans 0..nvirt-1
eps_v = eps[ndocc:]
print(oovv)

[[[[ 7.96127350e-01  1.94455779e-15]
   [ 2.03434755e-15  8.60293771e-01]]

  [[ 7.12356791e-03 -7.71384643e-18]
   [ 3.34599434e-17  9.24269491e-03]]

  [[-1.35253123e-16 -8.72552543e-03]
   [-8.72552543e-03  2.55736958e-16]]

  [[ 2.02684850e-02 -1.67895903e-16]
   [-1.44448023e-16  4.04803498e-03]]

  [[-5.13682111e-17 -4.22683408e-18]
   [-4.22683408e-18 -9.18082039e-17]]]


 [[[ 7.12356791e-03  2.02415350e-17]
   [ 2.39079648e-17  9.24269491e-03]]

  [[ 6.07755992e-01  4.85146545e-16]
   [ 4.25429750e-16  6.19456278e-01]]

  [[ 2.06813212e-15  9.53755410e-02]
   [ 9.53755410e-02 -3.89999869e-15]]

  [[-5.58316333e-02 -9.46413889e-18]
   [-4.42011876e-17  1.55984084e-02]]

  [[-2.68256472e-16  7.20649071e-17]
   [ 7.20649071e-17 -1.35714964e-16]]]


 [[[-1.23795714e-16 -8.72552543e-03]
   [-8.72552543e-03  2.55826799e-16]]

  [[ 2.10492756e-15  9.53755410e-02]
   [ 9.53755410e-02 -3.87433720e-15]]

  [[ 5.65071654e-01 -7.91638714e-16]
   [-7.88846729e-16  6.02287717e-01]]

  [[-1.9

In [44]:
Ham = np.zeros((ndocc*nvirt,ndocc*nvirt))

In [45]:
for i in range(0, ndocc):
    for a in range(0, nvirt):
        ia = i*nvirt + a
        
        for j in range(0, ndocc):
            for b in range(0, nvirt):
                jb = j*nvirt + b
                term1 = eps_v[a] - eps_o[i]
                term2 = 2 * ovov[i, a, j, b] - oovv[i,j,a,b]
                
                print(ia,jb, eps_v[a], eps_o[i], i, j, a, b, term1, term2)
                if (i==j) and (a == b):
                    Ham[ia, jb] = term1 + term2
                else:
                    Ham[ia, jb] = term2


0 0 0.5641522673084292 -20.247270330964387 0 0 0 0 20.811422598272816 -0.736773115795605
0 1 0.5641522673084292 -20.247270330964387 0 0 0 1 20.811422598272816 -2.2777806728915594e-15
0 2 0.5641522673084292 -20.247270330964387 0 1 0 0 20.811422598272816 -0.022555252094067713
0 3 0.5641522673084292 -20.247270330964387 0 1 0 1 20.811422598272816 -4.526129102547051e-16
0 4 0.5641522673084292 -20.247270330964387 0 2 0 0 20.811422598272816 1.5002344668795562e-16
0 5 0.5641522673084292 -20.247270330964387 0 2 0 1 20.811422598272816 0.021912164948562355
0 6 0.5641522673084292 -20.247270330964387 0 3 0 0 20.811422598272816 -0.0218140624064993
0 7 0.5641522673084292 -20.247270330964387 0 3 0 1 20.811422598272816 -5.479816414081163e-16
0 8 0.5641522673084292 -20.247270330964387 0 4 0 0 20.811422598272816 4.163920326323892e-17
0 9 0.5641522673084292 -20.247270330964387 0 4 0 1 20.811422598272816 1.1650923568731158e-17
1 0 0.6930072829137477 -20.247270330964387 0 0 1 0 20.940277613878134 -2.3642270

In [46]:
print(Ham)

[[ 2.00746495e+01 -2.27778067e-15 -2.25552521e-02 -4.52612910e-16
   1.50023447e-16  2.19121649e-02 -2.18140624e-02 -5.47981641e-16
   4.16392033e-17  1.16509236e-17]
 [-2.36422709e-15  2.01215885e+01 -6.18355639e-16 -4.64420474e-02
   1.53121833e-03 -8.64448827e-16 -1.52591891e-16 -2.92985188e-02
   2.52547569e-18  3.66837281e-17]
 [-2.25552521e-02 -6.04713341e-16  1.40388678e+00 -1.76728785e-15
  -4.39277847e-16  5.40915078e-02 -6.69481171e-02  1.03379885e-15
   2.07752643e-18  2.51414372e-17]
 [-5.03216872e-16 -4.64420474e-02 -1.88180683e-15  1.44877331e+00
  -2.74761460e-02  2.25462734e-16  2.06971237e-15  1.83042801e-02
  -8.81189612e-18  2.02412231e-16]
 [ 1.91190086e-16  1.53121833e-03 -5.62075147e-16 -2.74761460e-02
   7.39815212e-01 -2.51413439e-16 -2.87661514e-15 -4.28876392e-02
   2.03580767e-16 -7.87998408e-17]
 [ 2.19121649e-02 -8.71737086e-16  5.40915078e-02  2.14472070e-16
  -3.35380911e-16  9.95439096e-01 -1.17805212e-01 -6.41101852e-16
  -2.41923439e-16  3.08685471e-16

In [58]:
# diagonalize Hamiltonian
ECIS, CCIS = np.linalg.eig(Ham)
print(ECIS[8])
print(np.reshape(CCIS[:,8],(5,2)))
print(rpa)
#[[ 1.11496908e-16 -5.29257371e-20]
# [-5.20723397e-17 -1.62756122e-17]
# [-8.97175766e-16  1.09154215e-16]
# [-1.54415002e-15  2.95588487e-17]
# [ 1.00000000e+00  4.76559551e-16]]

0.442203017911775
[[-4.26948355e-18 -1.01537658e-17]
 [-1.09585646e-16 -3.26413497e-16]
 [-6.88848123e-16  1.12302237e-16]
 [-1.70828431e-15 -9.07220946e-16]
 [ 1.00000000e+00  2.34827205e-14]]
[{'EXCITATION ENERGY': 0.4422030179117806, 'ELECTRIC DIPOLE TRANSITION MOMENT (LEN)': array([ 1.03986362e-01, -1.16181470e-16, -7.06077139e-17]), 'OSCILLATOR STRENGTH (LEN)': 0.003187742366517262, 'ELECTRIC DIPOLE TRANSITION MOMENT (VEL)': array([ 1.36800051e-01, -2.26083283e-16,  1.60659272e-16]), 'OSCILLATOR STRENGTH (VEL)': 0.028213668239332943, 'MAGNETIC DIPOLE TRANSITION MOMENT': array([-7.97878720e-17, -2.63099938e-01, -8.05296591e-15]), 'ROTATORY STRENGTH (LEN)': 2.2270487024408488e-17, 'ROTATORY STRENGTH (VEL)': -1.098308033465929e-16, 'SYMMETRY': 'A', 'SPIN': 'singlet', 'RIGHT EIGENVECTOR ALPHA': <psi4.core.Matrix object at 0x7f88f051d8b0>, 'LEFT EIGENVECTOR ALPHA': <psi4.core.Matrix object at 0x7f88f051d8b0>, 'RIGHT EIGENVECTOR BETA': <psi4.core.Matrix object at 0x7f88f051d8b0>, 'LEFT 